# 統合分析結果の可視化

このノートブックでは、`unified_cv_analysis.py`と`unified_encoding_analysis.py`の結果を可視化します。

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

from src.const import ROOT_PATH

# 日本語フォント設定
plt.rcParams["font.family"] = ["BIZ UDPGothic", "Arial"]
sns.set_theme(font="BIZ UDPGothic")
sns.set_style("whitegrid")

## 1. 結果ファイルの読み込み

In [ ]:
# 最新の結果ファイルを自動検出
cv_results_dir = os.path.join(ROOT_PATH, "results", "unified_cv_analysis")
encoding_results_dir = os.path.join(ROOT_PATH, "results", "unified_encoding_analysis")

# 最新のJSONファイルを取得
cv_json_files = sorted(glob(os.path.join(cv_results_dir, "detailed_results_*.json")))
encoding_json_files = sorted(glob(os.path.join(encoding_results_dir, "encoding_results_*.json")))

if not cv_json_files or not encoding_json_files:
    print("結果ファイルが見つかりません。先にスクリプトを実行してください。")
else:
    latest_cv_json = cv_json_files[-1]
    latest_encoding_json = encoding_json_files[-1]
    
    print(f"CV結果: {latest_cv_json}")
    print(f"エンコーディング結果: {latest_encoding_json}")
    
    # JSONファイルを読み込み
    with open(latest_cv_json, 'r') as f:
        cv_results = json.load(f)
    
    with open(latest_encoding_json, 'r') as f:
        encoding_results = json.load(f)
    
    print("\n読み込み完了！")

## 2. グレーバーチャート（最終層の予測精度）

In [ ]:
# データの準備
plot_data = []
target_map = {
    "res_L": "好み"
}

for target_name, target_jp in target_map.items():
    for model_name, results in cv_results.items():
        if target_name in results:
            plot_data.append({
                "モデル": model_name,
                "属性": target_jp,
                "相関係数": results[target_name]["overall_score"]
            })

df = pd.DataFrame(plot_data)

# 各属性ごとにプロット
for target_jp in target_map.values():
    fig = plt.figure(figsize=(16, 9), dpi=300)
    target_df = df[df["属性"] == target_jp]
    
    # グレーのパレット
    n_models = len(target_df)
    gray_palette = [
        "#5E5F5F",  # 濃いグレー
        "#7D7D7D",  # 中程度のグレー
        "#959595",  # やや薄いグレー
        "#ADADAD"   # 薄いグレー
    ][:n_models]
    
    sns.barplot(
        data=target_df,
        x="モデル",
        y="相関係数",
        palette=gray_palette,
        width=0.6,
        errorbar=None
    )
    
    plt.xticks(fontsize=42, fontweight="bold")
    plt.xlabel("", fontsize=36)
    plt.yticks(fontsize=46, fontweight="bold")
    plt.ylabel("相関係数", fontsize=52, labelpad=20, fontweight="bold")
    plt.ylim(0.0, 1.0)
    plt.title(f"{target_jp}の予測精度", fontsize=48, fontweight="bold", pad=20)
    
    plt.tight_layout()
    plt.show()

## 3. 散布図（予測値 vs 実測値）

In [ ]:
# 注: 散布図を描くには、各foldの予測値と実測値が必要です。
# 現在のJSONファイルには含まれていないため、スクリプトを修正する必要があります。
print("散布図を描くには、予測値と実測値のデータが必要です。")
print("スクリプトを修正して、fold_y_tests と fold_y_preds も保存するようにしてください。")

## 4. エンコーディング分析結果（RSA_and_regression.ipynb風）

In [ ]:
# 各モデルごとに可視化
for model_name, layer_results in encoding_results.items():
    # データの整形
    plot_data = []
    for layer_idx, (layer_name, target_results) in enumerate(layer_results.items()):
        for target_name, score in target_results.items():
            plot_data.append({
                'Layer': layer_idx,
                'attribute': target_name,
                'Score': score
            })
    
    melt_df = pd.DataFrame(plot_data)
    
    # カテゴリ定義
    group_dict = {
        "Subjective value": ["res_L"],
        "Healthiness": ["res_H"],
        "Color (RGB)": ["R", "G", "B"],
        "Nutritional value": ["kcal_100g", "protein_100g", "fat_100g", "carbs_100g"],
    }
    
    # 各カテゴリの3層移動平均を計算
    df_list = []
    for key, attrs in group_dict.items():
        attr_df = pd.DataFrame(
            melt_df[melt_df["attribute"].isin(attrs)]
            .groupby("Layer")["Score"]
            .mean().sort_index()
            .rolling(window=3, min_periods=1, step=3)
            .mean()
            .dropna()
            .reset_index(drop=True)
        )
        attr_df["attr"] = key
        attr_df.index += 1
        df_list.append(attr_df)
    
    data = pd.concat(df_list)
    
    # プロット作成（2x2グリッド）
    fig, axes = plt.subplots(2, 2, figsize=(16, 9), dpi=300)
    colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]
    
    for i, (attr, d) in enumerate(data.groupby("attr")):
        ax = axes.flatten()[i]
        sns.lineplot(
            data=d.reset_index(),
            x="index",
            y="Score",
            color=colors[i],
            marker="o",
            markersize=15,
            linewidth=4,
            ax=ax
        )
        ax.set_ylim(0.15, 1)
        
        if i > 1:
            ax.set_xlabel("Layer (3-layer average)", fontsize=36, fontweight="bold")
            ax.set_xticklabels(
                range(0, int(d.index.max()) + 2, 2),
                fontsize=30,
                fontweight="bold",
            )
        else:
            ax.set_xlabel("")
            ax.set_xticklabels([])
        
        if i % 2 == 0:
            ax.set_ylabel("")
            labels = ax.get_yticklabels()
            new_labels = [label if j % 2 == 0 else "" for j, label in enumerate(labels)]
            ax.set_yticklabels(new_labels, fontsize=32, fontweight="bold")
        else:
            ax.set_ylabel("")
            ax.set_yticklabels([])
        
        ax.set_title(attr, fontsize=40, fontweight="bold", pad=-2)
    
    plt.tight_layout()
    
    # y軸ラベル（回転）
    plt.text(
        s="Explanatory power",
        x=-14.2,
        y=1,
        fontsize=42,
        fontweight="bold",
        va="center",
        rotation=90,
    )
    
    # タイトル
    plt.text(
        x=-4.2,
        y=2.27,
        s=f"Representation in {model_name}",
        fontsize=42,
        fontweight="bold",
        va="center",
    )
    
    plt.show()

## 5. エンコーディング分析のグレーバーチャート

In [ ]:
# データの準備
plot_data = []
target_map = {
    "res_L": "好み",
    "res_H": "健康度",
    "res_T": "美味しさ",
    "R": "赤",
    "G": "緑",
    "B": "青",
    "kcal_100g": "カロリー",
    "protein_100g": "タンパク質",
    "fat_100g": "脂質",
    "carbs_100g": "炭水化物"
}

category_map = {
    "res_L": "主観的価値",
    "res_H": "健康度",
    "res_T": "美味しさ",
    "R": "色",
    "G": "色",
    "B": "色",
    "kcal_100g": "栄養価",
    "protein_100g": "栄養価",
    "fat_100g": "栄養価",
    "carbs_100g": "栄養価"
}

for model_name, layer_results in encoding_results.items():
    for layer_name, target_results in layer_results.items():
        for target_name, score in target_results.items():
            if target_name in target_map:
                plot_data.append({
                    "モデル": model_name,
                    "属性": target_map[target_name],
                    "カテゴリ": category_map[target_name],
                    "スコア": score
                })

df = pd.DataFrame(plot_data)

# 各モデル×カテゴリの平均を計算
avg_df = df.groupby(["モデル", "カテゴリ"])["スコア"].mean().reset_index()

# カテゴリごとにプロット
for category in avg_df["カテゴリ"].unique():
    fig = plt.figure(figsize=(16, 9), dpi=300)
    category_df = avg_df[avg_df["カテゴリ"] == category]
    
    # グレーのパレット
    n_models = len(category_df)
    gray_palette = [
        "#5E5F5F",
        "#7D7D7D",
        "#959595",
        "#ADADAD"
    ][:n_models]
    
    sns.barplot(
        data=category_df,
        x="モデル",
        y="スコア",
        palette=gray_palette,
        width=0.6,
        errorbar=None
    )
    
    plt.xticks(fontsize=42, fontweight="bold")
    plt.xlabel("", fontsize=36)
    plt.yticks(fontsize=46, fontweight="bold")
    plt.ylabel("相関係数", fontsize=52, labelpad=20, fontweight="bold")
    plt.ylim(0.0, 1.0)
    plt.title(f"{category}の予測精度（層平均）", fontsize=48, fontweight="bold", pad=20)
    
    plt.tight_layout()
    plt.show()

## 6. 結果のサマリー

In [ ]:
# CV結果のサマリー
print("="*80)
print("最終層の予測精度")
print("="*80)

summary_data = []
for model_name, results in cv_results.items():
    row = {"モデル": model_name}
    for target_name in ["res_L"]:
        if target_name in results:
            row[target_map.get(target_name, target_name)] = f"{results[target_name]['overall_score']:.4f}"
    summary_data.append(row)

summary_df = pd.DataFrame(summary_data)
display(summary_df)

print("\n" + "="*80)
print("中間層の予測精度（平均）")
print("="*80)
display(avg_df.pivot(index="モデル", columns="カテゴリ", values="スコア"))